In [2]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
def prepare_sequence(seq, to_idx):
    idxs = [to_idx[w] for w in seq]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)

def prepare_chars(seq, to_idx):
    idxs = [to_idx[c] for w in seq for c in w]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)

training_data = [
    ('The dog ate the apple'.split(), ['DET', 'NN', 'V', 'DET', 'NN']),
    ('Everybody read that book'.split(), ['NN', 'V', 'DET', 'NN'])
]
word_to_idx = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_idx:
            word_to_idx[word] = len(word_to_idx)
print(word_to_idx)
tag_to_idx = {'DET': 0, 'NN': 1, 'V': 2}

char_to_idx = {}
for sent, tags in training_data:
    for word in sent:
        for char in word:
            if char not in char_to_idx:
                char_to_idx[char] = len(char_to_idx)
print(char_to_idx)

{'The': 0, 'dog': 1, 'Everybody': 5, 'apple': 4, 'ate': 2, 'the': 3, 'that': 7, 'book': 8, 'read': 6}
{'a': 6, 'b': 14, 'd': 3, 'r': 12, 'l': 9, 'y': 13, 'E': 10, 't': 7, 'o': 4, 'e': 2, 'T': 0, 'p': 8, 'v': 11, 'g': 5, 'k': 15, 'h': 1}


In [9]:
# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

class LSTMTagger(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.embeds = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer. Takes in the emdding_dim and outputs the output and hidden state with hidden_dims
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        
        # The linear layer that goes from the hidden_dim to the tag dim
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
                autograd.Variable(torch.zeros(1, 1, self.hidden_dim)))
    
    def forward(self, sentence):
        embeds = self.embeds(sentence)
        lstm_out, self.hidden = self.lstm(embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim = 1)
        return tag_scores

In [10]:
lstm_model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_idx), len(tag_to_idx))
loss_fn = nn.NLLLoss()
optimizer = optim.SGD(lstm_model.parameters(), lr = 0.1)

In [12]:
# Checking model
inputs = prepare_sequence(training_data[0][0], word_to_idx)
tag_scores = lstm_model(inputs)
print(tag_scores)

Variable containing:
-0.9891 -1.4370 -0.9405
-0.9828 -1.4138 -0.9610
-1.0182 -1.4499 -0.9060
-1.0130 -1.4432 -0.9145
-1.0858 -1.6048 -0.7735
[torch.FloatTensor of size 5x3]



In [17]:
# Train model
for epoch in range(300):
    for sentence, tags in training_data:
        # Clear out prev gradients and hidden states
        lstm_model.zero_grad()
        lstm_model.hidden = lstm_model.init_hidden()
        
        # Get sentences and target tags
        sentence_in = prepare_sequence(sentence, word_to_idx)
        targets = prepare_sequence(tags, tag_to_idx)
        
        # Get scores 
        tag_scores = lstm_model(sentence_in)
        
        # Backprop and update
        loss = loss_fn(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
inputs = prepare_sequence(training_data[0][0], word_to_idx)
tag_scores = lstm_model(inputs)
# The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
#  for word i. The predicted tag is the maximum scoring tag.
# Here, we can see the predicted sequence below is 0 1 2 0 1
# since 0 is index of the maximum value of row 1,
# 1 is the index of maximum value of row 2, etc.
# Which is DET NOUN VERB DET NOUN, the correct sequence!
print(tag_scores)

Variable containing:
-0.0150 -5.9274 -4.4041
-7.3409 -0.0017 -6.8742
-4.6537 -4.7948 -0.0180
-0.0060 -5.7922 -5.8216
-5.6614 -0.0042 -7.2186
[torch.FloatTensor of size 5x3]



In [22]:
# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
WORD_EMBEDDING_DIM = 6
CHAR_EMBEDDING_DIM = 6
HIDDEN_DIM = 6

class LSTMTaggerCharacterLevel(nn.Module):
    
    def __init__(self, word_embedding_dim, char_embedding_dim, hidden_dim, vocab_size, tagset_size, char_size):
        super(LSTMTaggerCharacterLevel, self).__init__()
        self.hidden_dim = hidden_dim
        self.char_hidden_dim = char_embedding_dim
        self.word_embeds = nn.Embedding(vocab_size, word_embedding_dim)
        
        # LSTM character embedding layer
        self.char_embeds = nn.Embedding(char_size, char_embedding_dim)
        self.char_lstm = nn.LSTM(char_embedding_dim, self.char_hidden_dim)
        
        # LSTM layer. Takes in the emdding_dim and outputs the output and hidden state with hidden_dims
        self.lstm = nn.LSTM(word_embedding_dim, hidden_dim)
        
        # The linear layer that goes from the hidden_dim to the tag dim
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()
        self.char_hidden = self.init_hidden_char()
        
    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
                autograd.Variable(torch.zeros(1, 1, self.hidden_dim)))
    
    def init_hidden_char(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (autograd.Variable(torch.zeros(1, 1, self.char_hidden_dim)),
                autograd.Variable(torch.zeros(1, 1, self.char_hidden_dim)))
    
    def forward(self, sentence):
        sent, chars = sentence
        word_embeds = self.word_embeds(sent)
        char_embeds = self.char_embeds(chars)
        
        char_lstm, self.char_hidden = self.char_lstm(char_embeds.view(len(chars), 1, -1), self.char_hidden)
        
        embeds = (word_embeds.view(len(sent), 1, -1), char_lstm)
        newEmbeds = torch.cat(embeds)
        #print(newEmbeds)
        lstm_out, self.hidden = self.lstm(newEmbeds, self.hidden)
        print(lstm_out.size())
        tag_space = self.hidden2tag(lstm_out.view(len(sent), -1))
        tag_scores = F.log_softmax(tag_space, dim = 1)
        return tag_scores

In [23]:
lstm_model = LSTMTaggerCharacterLevel(WORD_EMBEDDING_DIM, CHAR_EMBEDDING_DIM, HIDDEN_DIM, len(word_to_idx), len(tag_to_idx), len(char_to_idx))
loss_fn = nn.NLLLoss()
optimizer = optim.SGD(lstm_model.parameters(), lr = 0.1)

In [24]:
# Checking model
sentences = prepare_sequence(training_data[0][0], word_to_idx)
chars = prepare_chars(training_data[0][0], char_to_idx)
print(chars)
tag_scores = lstm_model((sentences, chars))
print(tag_scores)

Variable containing:
 0
 1
 2
 3
 4
 5
 6
 7
 2
 7
 1
 2
 6
 8
 8
 9
 2
[torch.LongTensor of size 17]

torch.Size([22, 1, 6])


RuntimeError: invalid argument 2: size '[5 x -1]' is invalid for input with 132 elements at /pytorch/torch/lib/TH/THStorage.c:37